<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>

<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in New Canada</font></h1>

In [3]:
#!pip install bs4

In [4]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup

print('Libraries imported.')


Libraries imported.


<a id='item1'></a>

## 1. Download and Explore Dataset

Use the Beautiful Soup for extract HTML.

In [5]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_url)
#print(soup.prettify())


Extract Table with Postal Code.


In [6]:
My_table = soup.find('table',{'class':'wikitable sortable'})

#### Tranform the data into a *pandas* dataframe
Get Table to Dataframe and Unassigned checks are made in Borough and Neighborhood

Clean Data Frame :)

In [23]:
Head = My_table.find_all('tr')
Table = []
for th in Head:
    row = np.array(th.getText()[1:-1].split('\n'))
    if row[1] != 'Not assigned': 
        if row[2] == 'Not assigned':
            row[2] = row[1]
        Table.append(row)
        
df_Canada = pd.DataFrame(data=Table[1:], columns=Table[0])
df_Canada.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


## 2. Trasnform Dataset

Get Name Columns

In [24]:
Col = df_Canada.columns
Col

Index(['Postcode', 'Borough', 'Neighbourhood'], dtype='object')

Group the neighborhood by Postcode

In [9]:
Col = df_Canada.columns
df_Canada_Group = df_Canada.groupby(Col[0])[Col[2]].apply(lambda x: ''.join([str(elem+',') for elem in list(x)])).reset_index()
df_Canada_Group.head()

,Postcode,Neighbourhood
0,M1B,"Rouge,Malvern,"
1,M1C,"Highland Creek,Rouge Hill,Port Union,"
2,M1E,"Guildwood,Morningside,West Hill,"
3,M1G,"Woburn,"
4,M1H,"Cedarbrae,"


Then let's loop through the data and fill the dataframe one row at a time.

In [12]:
df_repeated = df_Canada.sort_values('Postcode', ascending=True)
df_repeated.drop_duplicates('Postcode', keep="last", inplace=True)
df_Canada_Group['Borough'] = df_repeated.reset_index()['Borough']

In [13]:
df_Canada_Group.head()

,Postcode,Neighbourhood,Borough
0,M1B,"Rouge,Malvern,",Scarborough
1,M1C,"Highland Creek,Rouge Hill,Port Union,",Scarborough
2,M1E,"Guildwood,Morningside,West Hill,",Scarborough
3,M1G,"Woburn,",Scarborough
4,M1H,"Cedarbrae,",Scarborough


## 3. Geospatial data

In [14]:
#!pip install geocoder

Neighborhood has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood. 

Luckily, this dataset exists for free on the web. Feel free to try to find this dataset on your own, but here is the link to the dataset: https://geo.nyu.edu/catalog/nyu_2451_34572

For your convenience, I downloaded the files and placed it on the server, so you can simply run a `wget` command and access the data. So let's go ahead and do that.

In [15]:
!wget -q -O 'postal_data.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


#### Load and explore the data

Next, let's load the data.

In [16]:
df_csv = pd.read_csv('postal_data.csv')
df_csv.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [25]:
df_Canada_Group['Latitude'] = df_csv['Latitude']
df_Canada_Group['Longitude'] = df_csv['Longitude']
df_Canada_Group.head()

,Postcode,Neighbourhood,Borough,Latitude,Longitude
0,M1B,"Rouge,Malvern,",Scarborough,43.806686,-79.194353
1,M1C,"Highland Creek,Rouge Hill,Port Union,",Scarborough,43.784535,-79.160497
2,M1E,"Guildwood,Morningside,West Hill,",Scarborough,43.763573,-79.188711
3,M1G,"Woburn,",Scarborough,43.770992,-79.216917
4,M1H,"Cedarbrae,",Scarborough,43.773136,-79.239476


## 3. Visualization

And make sure that the dataset has all 5 boroughs and 306 neighborhoods.

#### Use geopy library to get the latitude and longitude values of Toronto.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>tr_explorer</em>, as shown below.

In [18]:
address = 'Toronto, YYZ'

geolocator = Nominatim(user_agent="tr_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.678524, -79.6291291306445.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [35]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_Canada_Group['Latitude'], df_Canada_Group['Longitude'], df_Canada_Group['Borough'], df_Canada_Group['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='#5A99A2',
        fill=True,
        fill_color='#3AC2CE',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

**Folium** is a great visualization library. Feel free to zoom into the above map, and click on each circle mark to reveal the name of the neighborhood and its respective borough.

However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in Scarborough. So let's slice the original dataframe and create a new dataframe of the Scarborough data.

In [20]:
scarborough_data = df_Canada_Group[df_Canada_Group['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head()

,Postcode,Neighbourhood,Borough,Latitude,Longitude
0,M1B,"Rouge,Malvern,",Scarborough,43.806686,-79.194353
1,M1C,"Highland Creek,Rouge Hill,Port Union,",Scarborough,43.784535,-79.160497
2,M1E,"Guildwood,Morningside,West Hill,",Scarborough,43.763573,-79.188711
3,M1G,"Woburn,",Scarborough,43.770992,-79.216917
4,M1H,"Cedarbrae,",Scarborough,43.773136,-79.239476


In [56]:
# define the dataframe columns
column_names = ['Postcode', 'Borough', 'Neighbourhood','Latitude','Longitude']

# instantiate the dataframe
neighborhoods_scar = pd.DataFrame(columns=column_names)

# Iteración por filas del DataFrame:
for idx_row, row in scarborough_data.iterrows():
    Postal_Code = row[0]
    Borough = row[2]
    Latitude = row[3]
    Longitude = row[4]
    for Neighbourhood in row[1].split(','):
        if Neighbourhood !='':
            neighborhoods_scar = neighborhoods_scar.append({'Postcode':Postal_Code,'Borough':Borough,'Neighbourhood':Neighbourhood,
                                                            'Latitude':Latitude,'Longitude':Longitude}, ignore_index=True)
neighborhoods_scar.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,Rouge,43.806686,-79.194353
1,M1B,Scarborough,Malvern,43.806686,-79.194353
2,M1C,Scarborough,Highland Creek,43.784535,-79.160497
3,M1C,Scarborough,Rouge Hill,43.784535,-79.160497
4,M1C,Scarborough,Port Union,43.784535,-79.160497


Let's get the geographical coordinates of Scarborough.

In [57]:
address = 'Scarborough,Toronto'

geolocator = Nominatim(user_agent="sb_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


As we did with all of Scarborough, let's visualizat Scarborough the neighborhoods in it.

In [81]:
# create map of Toronto using latitude and longitude values
map_scarborough = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(neighborhoods_scar['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='#5A99A2',
        fill=True,
        fill_color='#3AC2CE',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)  
    
map_scarborough

#### Define Foursquare Credentials and Version

In [66]:
CLIENT_ID = '414XZANJAD1HTUW5OGCCVYVBZINSZMLI1TLOP4HRLMSLV1M3' # your Foursquare ID
CLIENT_SECRET = '4Z4UIKJKZEXQ30I05L0JY1YM4GXYD3OD22IPCJ33P4DX2GPU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 414XZANJAD1HTUW5OGCCVYVBZINSZMLI1TLOP4HRLMSLV1M3
CLIENT_SECRET:4Z4UIKJKZEXQ30I05L0JY1YM4GXYD3OD22IPCJ33P4DX2GPU


Get the neighborhood's latitude and longitude values.

In [67]:
neighborhood_latitude = neighborhoods_scar.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods_scar.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = neighborhoods_scar.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge are 43.806686299999996, -79.19435340000001.


#### Now, let's get the top 100 venues that are in Rouge within a radius of 500 meters.

First, let's create the GET request URL. Name your URL **url**.

In [73]:
# type your answer here

LIMIT = 1000 # limit of number of venues returned by Foursquare API
radius = 5000 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=414XZANJAD1HTUW5OGCCVYVBZINSZMLI1TLOP4HRLMSLV1M3&client_secret=4Z4UIKJKZEXQ30I05L0JY1YM4GXYD3OD22IPCJ33P4DX2GPU&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=5000&limit=1000'

Send the GET request and examine the resutls

In [80]:
results = requests.get(url).json()
#results

From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [75]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [78]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Toronto Pan Am Sports Centre,Athletics & Sports,43.790623,-79.193869
1,African Rainforest Pavilion,Zoo Exhibit,43.817725,-79.183433
2,Toronto Zoo,Zoo,43.820582,-79.181551
3,Polar Bear Exhibit,Zoo,43.823372,-79.185145
4,Australasia Pavillion,Zoo Exhibit,43.822563,-79.183286


And how many venues were returned by Foursquare?

In [79]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


## 2. Explore Neighborhoods in Toronto

#### Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [82]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *Scarborough_venues*.

In [85]:
# type your answer here

Scarborough_venues = getNearbyVenues(names=neighborhoods_scar['Neighbourhood'],
                                   latitudes=neighborhoods_scar['Latitude'],
                                   longitudes=neighborhoods_scar['Longitude']
                                  )



Rouge
Malvern
Highland Creek
Rouge Hill
Port Union
Guildwood
Morningside
West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park
Ionview
Kennedy Park
Clairlea
Golden Mile
Oakridge
Cliffcrest
Cliffside
Scarborough Village West
Birch Cliff
Cliffside West
Dorset Park
Scarborough Town Centre
Wexford Heights
Maryvale
Wexford
Agincourt
Clarks Corners
Sullivan
Tam O'Shanter
Agincourt North
L'Amoreaux East
Milliken
Steeles East
L'Amoreaux West
Upper Rouge


Double-click __here__ for the solution.
<!-- The correct answer is:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )
--> 

#### Let's check the size of the resulting dataframe

In [87]:
print(Scarborough_venues.shape)
Scarborough_venues.head()

(183, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rouge,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,Malvern,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
2,Highland Creek,43.784535,-79.160497,RIGHT WAY TO GOLF,43.785177,-79.161108,Golf Course
3,Highland Creek,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,Rouge Hill,43.784535,-79.160497,RIGHT WAY TO GOLF,43.785177,-79.161108,Golf Course


Let's check how many venues were returned for each neighborhood

In [88]:
Scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
Agincourt North,2,2,2,2,2,2
Birch Cliff,4,4,4,4,4,4
Cedarbrae,9,9,9,9,9,9
Clairlea,8,8,8,8,8,8
Clarks Corners,11,11,11,11,11,11
Cliffcrest,2,2,2,2,2,2
Cliffside,2,2,2,2,2,2
Cliffside West,4,4,4,4,4,4


#### Let's find out how many unique categories can be curated from all the returned venues

In [89]:
print('There are {} uniques categories.'.format(len(Scarborough_venues['Venue Category'].unique())))

There are 51 uniques categories.


<a id='item3'></a>

## 3. Analyze Each Neighborhood

In [90]:
# one hot encoding
Scarborough_onehot = pd.get_dummies(Scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Scarborough_onehot['Neighborhood'] = Scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Scarborough_onehot.columns[-1]] + list(Scarborough_onehot.columns[:-1])
Scarborough_onehot = Scarborough_onehot[fixed_columns]

Scarborough_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Brewery,Bubble Tea Shop,Bus Line,Café,Camera Store,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Gas Station,General Entertainment,Golf Course,Grocery Store,Hakka Restaurant,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Thai Restaurant,Vietnamese Restaurant
0,Rouge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Malvern,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Highland Creek,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Highland Creek,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Rouge Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [91]:
Scarborough_onehot.shape

(183, 52)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [92]:
Scarborough_grouped = Scarborough_onehot.groupby('Neighborhood').mean().reset_index()
Scarborough_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Brewery,Bubble Tea Shop,Bus Line,Café,Camera Store,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Gas Station,General Entertainment,Golf Course,Grocery Store,Hakka Restaurant,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Thai Restaurant,Vietnamese Restaurant
0,Agincourt,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.250000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.250000,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.250000,0.000000,0.00,0.000,0.000000,0.000000
1,Agincourt North,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.500,0.000000,0.000000,0.000000,0.5,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.000000
2,Birch Cliff,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.25,0.000000,0.000000,0.000000,0.000000,0.25,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.25,0.000,0.000000,0.000000
3,Cedarbrae,0.0,0.111111,0.000000,0.111111,0.111111,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.111111,0.000000,0.000000,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.111111,0.111111,0.00,0.0,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.00,0.111111,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000,0.111111,0.000000
4,Clairlea,0.0,0.000000,0.000000,0.125000,0.000000,0.0,0.000000,0.000000,0.000000,0.25,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.00,0.000000,0.000000,0.125000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.00,0.000000,0.000000,0.125,0.000000,0.000000,0.0,0.000000,0.125,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.125,0.000000,0.000000
5,Clarks Corners,0.0,0.000000,0.000000,0.000000,0.090909,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.090909,0.000000,0.00,0.0,0.00,0.000000,0.000000,0.090909,0.090909,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.090909,0.000,0.000000,0.090909,0.181818,0.0,0.090909,0.000000,0.000000,0.00,0.000,0.090909,0.000000
6,Cliffcrest,0.5,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.5,0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.000000
7,Cliffside,0.5,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.5,0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.000000


#### Let's confirm the new size

In [93]:
Scarborough_grouped.shape

(36, 52)

#### Let's print each neighborhood along with the top 5 most common venues

In [95]:
num_top_venues = 5

for hood in Scarborough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Scarborough_grouped[Scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant  0.25
1             Breakfast Spot  0.25
2             Sandwich Place  0.25
3                     Lounge  0.25
4        American Restaurant  0.00


----Agincourt North----
                venue  freq
0                Park   0.5
1          Playground   0.5
2        Noodle House   0.0
3        Intersection   0.0
4  Italian Restaurant   0.0


----Birch Cliff----
                   venue  freq
0        College Stadium  0.25
1           Skating Rink  0.25
2  General Entertainment  0.25
3                   Café  0.25
4    American Restaurant  0.00


----Cedarbrae----
                venue  freq
0     Thai Restaurant  0.11
1              Bakery  0.11
2                Bank  0.11
3    Hakka Restaurant  0.11
4  Athletics & Sports  0.11


----Clairlea----
                  venue  freq
0              Bus Line  0.25
1          Intersection  0.12
2                Bakery  0.12
3          Soccer Field  0.12
4  Fast Food 

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [96]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [97]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Scarborough_grouped['Neighborhood']

for ind in np.arange(Scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Sandwich Place,Breakfast Spot,Latin American Restaurant,Lounge,Vietnamese Restaurant,Chinese Restaurant,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
1,Agincourt North,Playground,Park,Vietnamese Restaurant,Camera Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store
2,Birch Cliff,General Entertainment,Skating Rink,College Stadium,Café,Caribbean Restaurant,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
3,Cedarbrae,Caribbean Restaurant,Lounge,Thai Restaurant,Hakka Restaurant,Fried Chicken Joint,Gas Station,Athletics & Sports,Bakery,Bank,Bar
4,Clairlea,Bus Line,Metro Station,Soccer Field,Fast Food Restaurant,Bakery,Intersection,Park,Vietnamese Restaurant,Electronics Store,Discount Store
